In [1]:
import pandas as pd
import numpy as np
import pycountry
from scipy.optimize import least_squares
import random
import statsmodels
from scipy.optimize import minimize
from scipy.optimize import fsolve
#from pandas.core import datetools
import statsmodels.api as sm
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from collections import defaultdict
from math import pi, e
import model_functions as mf

/home/sara/myenv/lib64/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
resident_foreigners_norm = pd.read_table("/home/sara/Documents/Immigration/Shared_statistics/Data_final/resident_foreigners_norm.csv")
# Untill 2011 - in 2011 change
years = list(range(2004, 2008))

In [3]:
#reg_cap_info = pd.read_table("/home/sara/Documents/Immigration/Data_not_git/Prov_info/reg_cap_info.csv")
#regions_cap = sorted(reg_cap_info["Prov Capitals"].values)
reg_cap_info = pd.read_table("/home/sara/Documents/Immigration/Data_not_git/Prov_info/prov_cap_info.csv", index_col=[0])

In [4]:
regions = pd.read_table("/home/sara/Documents/Immigration/Shared_statistics/Data_final/regioni.csv")
regions_info = pd.read_table("/home/sara/Documents/Immigration/Shared_statistics/Data_final/region_info.csv")

mezzogiorno = {'Abruzzo': 1, 'Lazio': 0, 'Umbria': 0, 'Provincia Autonoma Trento': 0, 
               'Friuli-Venezia Giulia': 0, 'Molise': 1, 'Calabria': 1, 
               "Valle d'Aosta / Vallée d'Aoste": 0, 'Lombardia': 0, 'Liguria': 0, 
               'Emilia-Romagna': 0, 'Sicilia': 1, 
               'Provincia Autonoma Bolzano / Bozen': 0, 'Puglia': 1, 'Campania': 1, 
               'Piemonte': 0, 'Toscana': 0, 'Sardegna': 1, 'Marche': 0, 
               'Basilicata': 1, 'Veneto': 0}

In [5]:
x_df = pd.DataFrame()
#x_df = reg_cap_info[["Prov Capitals", "Area", "Dens"]].copy()
x_df = reg_cap_info[["Province", "Area", "Density"]].copy()
# Logarithmic transformation
x_df["Area"] = np.log(x_df["Area"])
x_df["Density"] = np.log(x_df["Density"])
#x_df["Mezzogiorno"] = [mezzogiorno[regions[regions["Provincia"] == i]["Regione"].values[0]] for i in x_df["Prov Capitals"].values]

In [6]:
x_df = x_df.set_index(["Province"])
x_df = x_df.sort_index()

In [7]:
x_df.head()

,Area,Density
Province,,
Agrigento,8.023746,4.976734
Alessandria,8.177187,4.787492
Ancona,7.582341,5.488938
Arezzo,8.081191,4.672829
Ascoli Piceno,7.113362,5.141664


In [8]:
data = resident_foreigners_norm.copy()
data = data.groupby(["Province", "Country", "Year"], as_index=False).sum()

In [9]:
data.head()

,Province,Country,Year,Value
0,Agrigento,AFG,2008,2
1,Agrigento,AFG,2009,12
2,Agrigento,AFG,2010,51
3,Agrigento,AFG,2011,35
4,Agrigento,AFG,2012,21


In [10]:
terr = sorted(list(set(resident_foreigners_norm["Province"])))
not_103 = ["Ogliastra", "Monza e della Brianza", "Medio Campidano", "Fermo", 
           "Carbonia-Iglesias", "Barletta-Andria-Trani", "Olbia-Tempio"]
terr = [i for i in terr if i not in not_103]

In [11]:
# Distance matrix related to the interested locations (regions capitals)
temp_W = pd.read_table("/home/sara/Documents/Immigration/Shared_models/Data/Distances_matrix.csv", sep = "\t", index_col=0).loc[terr][terr]

In [12]:
temp_W.head()

,Agrigento,Alessandria,Ancona,Arezzo,Ascoli Piceno,Asti,Avellino,Bari,Belluno,Benevento,...,Udine,Valle d'Aosta / Vallée d'Aoste,Varese,Venezia,Verbano-Cusio-Ossola,Vercelli,Verona,Vibo Valentia,Vicenza,Viterbo
Agrigento,0.00,1.520282e+06,1.154407e+06,572277.040000,1.089223e+06,1.552215e+06,731677.380000,703529.240000,572277.04000,768994.580000,...,1.576328e+06,572277.040000,1.569916e+06,572277.040000,1.597015e+06,1.569912e+06,572277.04000,3.619691e+05,572277.040000,1.037766e+06
Alessandria,1520281.89,0.000000e+00,4.573027e+05,385366.497500,5.681687e+05,3.763389e+04,810608.562500,913715.338125,434130.11125,790890.543125,...,4.573015e+05,167729.035625,1.419872e+05,346190.763750,1.418589e+05,5.552813e+04,235796.18125,1.173750e+06,285590.234375,5.228014e+05
Ancona,1154406.75,4.573027e+05,0.000000e+00,242641.575625,1.141482e+05,4.907148e+05,438618.041875,456566.656875,447901.06250,407252.399375,...,4.734930e+05,608873.778750,4.847525e+05,359845.529375,5.120943e+05,5.040044e+05,354842.31500,8.105737e+05,356127.051875,2.474618e+05
Arezzo,572277.04,3.853665e+05,2.426416e+05,0.000000,2.858283e+05,4.210109e+05,436974.663125,631590.724375,420049.72125,417501.710625,...,4.456417e+05,550710.037500,4.368461e+05,331994.182500,4.651546e+05,4.387711e+05,307902.62250,7.997079e+05,328275.710625,1.477071e+05
Ascoli Piceno,1089222.97,5.681687e+05,1.141482e+05,285828.296250,0.000000e+00,6.013101e+05,373114.746875,389874.518125,558496.29375,341749.104375,...,5.840883e+05,719469.010000,5.937786e+05,470440.760625,6.226896e+05,6.145996e+05,465437.54625,7.450704e+05,466722.283125,2.327132e+05


In [13]:
temp_W = (1/temp_W)**2
# w_ij = 0 if i=j
temp_W[temp_W == np.inf] = 0
# row standardization: every row sum up to 1
temp_W = temp_W.div(temp_W.sum(axis=1), axis=0)

Given a spatial weights matrix W is a nonnegative matrix with $w_{ij} >= 0$ and $w_{ii} = 0$. W uses to be symmetric.

The row-normalized W is used for ease of interpretation. It is defined as $\sum_{j=1}^n w_{ij} = 1, \forall i = 1, \dots, n$. This ensure that all weights are between 0 and 1.

Each rownormalized weight, $wij$, can be interpreted as the fraction of all spatial influence on unit $i$ attributable to unit $j$.

In [14]:
temp_W.head()

,Agrigento,Alessandria,Ancona,Arezzo,Ascoli Piceno,Asti,Avellino,Bari,Belluno,Benevento,...,Udine,Valle d'Aosta / Vallée d'Aoste,Varese,Venezia,Verbano-Cusio-Ossola,Vercelli,Verona,Vibo Valentia,Vicenza,Viterbo
Agrigento,0.000000,0.000502,0.000871,0.003545,0.000979,0.000482,0.002169,0.002346,0.003545,0.001963,...,0.000467,0.003545,0.000471,0.003545,0.000455,0.000471,0.003545,0.008862,0.003545,0.001078
Alessandria,0.000137,0.000000,0.001511,0.002127,0.000979,0.223040,0.000481,0.000378,0.001676,0.000505,...,0.001511,0.011229,0.015669,0.002636,0.015697,0.102451,0.005682,0.000229,0.003873,0.001156
Ancona,0.000429,0.002735,0.000000,0.009716,0.043903,0.002376,0.002973,0.002744,0.002851,0.003449,...,0.002552,0.001543,0.002434,0.004418,0.002181,0.002252,0.004543,0.000871,0.004510,0.009342
Arezzo,0.001638,0.003613,0.009114,0.000000,0.006568,0.003027,0.002810,0.001345,0.003041,0.003078,...,0.002702,0.001769,0.002812,0.004868,0.002480,0.002787,0.005660,0.000839,0.004979,0.024594
Ascoli Piceno,0.000411,0.001512,0.037459,0.005974,0.000000,0.001350,0.003506,0.003211,0.001565,0.004179,...,0.001431,0.000943,0.001384,0.002205,0.001259,0.001292,0.002253,0.000879,0.002241,0.009013


## Step I

In [15]:
def stepI(param, data_, W, times, ref_I, territories):
    beta = param[0]
    a = param[1:-1]
    ro = param[-1]
    
    T = len(times)
    I = len(territories)
    
    identity_I = np.identity(I)
    identity_I_1 = np.identity(I-1)
    neg1 = np.negative(np.ones((I-1, 1)))
    # Not-squared matrix
    Q = np.append(identity_I_1, neg1, axis=1)
    # All the I-1 locations (all but the reference one)
    terr_not_ref = [i for i in territories if i != ref_I]
    
    # Modify W s.t. the "ref_I" location is the last one (so that Q is well defined)
    W = W.reindex(index = terr_not_ref+[ref_I], columns = terr_not_ref+[ref_I])
     
    # Time-invariant quantity
    L = Q.dot(np.linalg.inv(identity_I-ro*W)).dot(np.linalg.inv(identity_I-ro*W.T)).dot(Q.T)
    
    log_lik = T*np.log(np.linalg.det(L))
    
    for t in times[1:]:
        y = data_.loc[(t, terr_not_ref), "Value"].values/data_.loc[t].loc[ref_I].values
        x = data_.loc[(t-1, terr_not_ref), "Value"].values/data_.loc[t-1].loc[ref_I].values
        #print(y.shape, x.shape, len(a))
        main_term = np.log(y) - beta*np.log(x) - a
        
        log_lik += main_term.T.dot(np.linalg.inv(L)).dot(main_term)
        
    return(log_lik)

## Step II

In [16]:
def stepII(theta, a, x_, ref_I, territories):
    # All the I-1 locations (all but the reference one)
    terr_not_ref = [i for i in territories if i != ref_I]
    
    x_I = x_.loc[ref_I].values
    temp = np.array([(a[terr_not_ref.index(i)] - np.dot(np.subtract(x_.loc[i].values, x_I), theta)) for i in terr_not_ref])
    
    log_lik = temp.T.dot(temp)
    return(log_lik)

In [17]:
'''def stepII(param, a, x_, ref_I, territories):
    theta = param[:-1]
    c = param[-1]
    # All the I-1 locations (all but the reference one)
    terr_not_ref = [i for i in territories if i != ref_I]
    
    x_I = x_.loc[ref_I].values
    temp = np.array([(a[terr_not_ref.index(i)] - np.dot(np.subtract(x_.loc[i].values, x_I), theta) - c) for i in terr_not_ref])
    
    log_lik = temp.T.dot(temp)
    return(log_lik)'''

'def stepII(param, a, x_, ref_I, territories):\n    theta = param[:-1]\n    c = param[-1]\n    # All the I-1 locations (all but the reference one)\n    terr_not_ref = [i for i in territories if i != ref_I]\n    \n    x_I = x_.loc[ref_I].values\n    temp = np.array([(a[terr_not_ref.index(i)] - np.dot(np.subtract(x_.loc[i].values, x_I), theta) - c) for i in terr_not_ref])\n    \n    log_lik = temp.T.dot(temp)\n    return(log_lik)'

In [18]:
data_all, country, times, I, x_ = data, "Romania", list(range(2004, 2008)), "Roma", x_df


territories = terr

data_all = data_all[data_all["Year"].isin(times)]
missing_territories = mf.not_including(data_all, times, territories)
territories = [i for i in territories if i not in missing_territories]
data_all = data_all[data_all["Province"].isin(territories)]

# Also the stock in the refered province is needed in the optimization 
data_ = data_all[data_all["Country"] == pycountry.countries.get(name=country).alpha_3]
del data_["Country"]

data_ = pd.DataFrame(data_.groupby(["Year", "Province"])["Value"].sum())


print("---------- Step I ----------")
initial_time = datetime.datetime.now()
print ("Current time: " + str(initial_time.strftime('%H:%M:%S') ))

# I-1 locations + beta + ro
random.seed(123)
param_init = [np.random.random() for i in range(len(territories)+1)]
#param_init = np.random.rand(len(territories)+1)
res_stepI =  minimize(stepI, param_init, args = (data_, temp_W, times, I, territories), method='CG')
print(res_stepI.message)

final_time = datetime.datetime.now() 
print ("Current time: " + str(final_time.strftime('%H:%M:%S')))
print("Computational time: " + str((final_time - initial_time)))


# Step I results and validation
beta_hat = res_stepI.x[0]
a_hat = res_stepI.x[1:-1]
rho_hat = res_stepI[-1]

---------- Step I ----------
Current time: 09:07:07


KeyboardInterrupt: 

In [28]:
print(beta_hat)
print(rho_hat)

0.3685728202528231
0.08007090426589598


In [29]:
res_stepI

     fun: 19.523959388110647
     jac: array([-2.14576721e-06, -1.66893005e-06, -7.15255737e-07, -7.15255737e-07,
       -4.76837158e-07, -9.53674316e-07, -9.53674316e-07, -1.19209290e-06,
       -1.43051147e-06, -1.43051147e-06, -1.90734863e-06, -7.15255737e-07,
       -1.19209290e-06, -9.53674316e-07, -1.43051147e-06, -4.76837158e-07,
       -1.90734863e-06, -1.90734863e-06, -1.90734863e-06, -1.90734863e-06,
       -1.43051147e-06, -1.66893005e-06, -1.90734863e-06,  4.76837158e-07,
       -7.15255737e-07, -1.66893005e-06, -2.38418579e-07, -2.14576721e-06,
       -7.15255737e-07, -2.14576721e-06, -1.43051147e-06, -4.76837158e-07,
       -1.43051147e-06, -9.53674316e-07, -7.15255737e-07, -9.53674316e-07,
       -1.66893005e-06, -9.53674316e-07, -1.43051147e-06, -1.66893005e-06,
       -9.53674316e-07, -1.19209290e-06, -4.76837158e-07, -1.43051147e-06,
       -9.53674316e-07, -1.19209290e-06, -9.53674316e-07, -7.15255737e-07,
       -7.15255737e-07, -9.53674316e-07, -1.43051147e-06, -1.

# Run the different models for the different origin country

In [172]:
def run_model(data_all, country, times, I, x_, territories = None):
    if not territories:
        territories = sorted(list(set(data_all["Province"])))
        
    data_all = data_all[data_all["Year"].isin(times)]
    missing_territories = mf.not_including(data_all, times, territories)
    territories = [i for i in territories if i not in missing_territories]
    data_all = data_all[data_all["Province"].isin(territories)]

    # Also the stock in the refered province is needed in the optimization 
    data_ = data_all[data_all["Country"] == pycountry.countries.get(name=country).alpha_3]
    del data_["Country"]

    data_ = pd.DataFrame(data_.groupby(["Year", "Province"])["Value"].sum())

    print("---------- Step I ----------")
    initial_time = datetime.datetime.now()
    print ("Current time: " + str(initial_time.strftime('%H:%M:%S') ))

    # I-1 locations + beta + ro
    random.seed(123)
    param_init = [np.random.random() for i in range(len(territories))]
    #param_init = np.random.rand(len(territories)+1)
    res_stepI =  minimize(stepI, param_init, args = (data_, temp_W, times, I, territories), method='CG')
    print(res_stepI.message)

    final_time = datetime.datetime.now() 
    print ("Current time: " + str(final_time.strftime('%H:%M:%S')))
    print("Computational time: " + str((final_time - initial_time)))
    
    
    # Step I results and validation
    beta_hat = res_stepI.x[0]
    a_hat = res_stepI.x[1:]
    '''y_hat = []
    y = []
    for i in territories:
        time_invariant = a_hat[territories.index(i)]
        for t in times[1:]:
            y.append(np.log(n_it(data_, i, t)/n_it(data_, I, t)))
            y_hat.append(beta_hat*(np.log(n_it(data_, i, t-1)/n_it(data_, I, t-1))) + time_invariant)
            
    y_mean = np.mean(y)

    R2 = 1 - sum(np.subtract(y, y_hat)**2) / sum((y - y_mean)**2)
    # Equivalently: 1 - (res_stepI.fun / sum((y - y_mean)**2))
    #print(R2)
    print("The R2 score from the step I is: %f" %R2)
    '''
    #print(beta_hat, a_hat)
    print("---------- Step II ----------")

    initial_time = datetime.datetime.now()
    print ("Current time: " + str(initial_time.strftime('%H:%M:%S') ))

    random.seed(123)
    param_init = [np.random.random() for i in range(len(x_.columns))]
    #param_init = np.random.uniform(0, 1, len(x_df.columns)-1)
    res_stepII =  minimize(stepII, param_init, args = (a_hat, x_, I, territories), method='CG')
    #print(model_I([b, a], data_rou), b, a)
    #print(res_stepII.x)
    print(res_stepII.message)
    #print(res_stepII.fun)
    final_time = datetime.datetime.now() 
    print ("Current time: " + str(final_time.strftime('%H:%M:%S')))
    print("Computational time: " + str((final_time - initial_time)))
    
    # Step II results and validation
    theta_hat = res_stepII.x
    '''x_I = x_[x_["Prov Capitals"] == I][["Area", "Dens", "Mezzogiorno"]].values
    y_hat = []
    #y = []
    for i in territories:
        x_i = x_[x_["Prov Capitals"] == i][["Area", "Dens", "Mezzogiorno"]].values
        time_invariant = np.dot(np.subtract(x_i, x_I), theta_hat)
        for t in times[1:]:
            #y.append(np.log(n_it(data_, i, t)/n_it(data_, I, t)))
            y_hat.append((beta_hat*(np.log(n_it(data_, i, t-1)/n_it(data_, I, t-1))) + time_invariant)[0])
            
    #y_mean = np.mean(y)
    R2 = 1 - sum(np.subtract(y, y_hat)**2) / sum((y - y_mean)**2)
    #print(R2)
    print("The final R2 score is: %f" %R2)
    
    n = len(y)
    k = len(x_.columns)-1
    R2_adj = 1 - (1 - R2)*((n - 1)/(n - k -1))
    print("The final Adjusted R2 score is: %f" %R2_adj)'''
    
    return(beta_hat, a_hat, theta_hat)

In [173]:
res_rou = run_model(data, "Romania", list(range(2004, 2008)), "Roma", x_df)

---------- Step I ----------
Current time: 16:14:03
Desired error not necessarily achieved due to precision loss.
Current time: 16:25:39
Computational time: 0:11:35.605179
---------- Step II ----------
Current time: 16:25:39


ValueError: setting an array element with a sequence.

In [ ]:
print("Beta parameter: %f" %res_rou[0])
print(res_rou[1])
#print("Theta parameter: %s %s %s" %tuple(res_rou[2][0]))
print(res_rou[2])

In [ ]:
res_mor = run_model(data, "Morocco", list(range(2004, 2008)), "Roma", x_df)

In [ ]:
print("Beta parameter: %f" %res_mor[0])
print(res_mor[1])
print("Theta parameter: %s %s %s" %tuple(res_mor[2]))